<a href="https://colab.research.google.com/github/Pratham-ja/SQL-PROJECT-/blob/main/Tokenizer_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
vocab = tokenizer.get_vocab()
print("Vocab Size:", len(vocab))
print(type(vocab))

Vocab Size: 30522
<class 'dict'>


In [ ]:
out = vocab['i']
print(out)

1045


In [ ]:
print(tokenizer("i am pratham"))

{'input_ids': [101, 1045, 2572, 10975, 8988, 3286, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


In [ ]:
## various methods related to tokenizer object are
tokens = tokenizer.tokenize("i am pratham")
print(tokens)

['i', 'am', 'pr', '##ath', '##am']


In [ ]:
input_ids = [tokenizer.convert_tokens_to_ids(token)for token in tokens]
input_ids

[1045, 2572, 10975, 8988, 3286]

In [ ]:
encoded = tokenizer.encode("i am pratham")
print(encoded)

[101, 1045, 2572, 10975, 8988, 3286, 102]


In [ ]:
decode = tokenizer.decode(encoded)
print(decode)

[CLS] i am pratham [SEP]


In [ ]:
print(vocab['[CLS]'])

101


Each model has corresponding padding token which is used to pad the ids to make them of equal length because a tensor array must have the same length and it sshould be a rectangular in shape

In [ ]:
tokenizer.pad_token_id

0

Now I am gonna show that when a single sentence is converted to input_id it is different than whan it is batched and sent with other sentences


In [56]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
sentences = ["My name is pratham","I live in India"]
tokens = [tokenizer.tokenize(sentence) for sentence in sentences]
ids = [tokenizer.convert_tokens_to_ids(token)for token in tokens]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [57]:
print(ids[0])
print(ids[1])

[2026, 2171, 2003, 10975, 8988, 3286]
[1045, 2444, 1999, 2634]


In [58]:
import torch

First see that if i convert this list of ids to tensor it will be an error

In [ ]:
input_ids = torch.tensor(ids)

ValueError: expected sequence of length 6 at dim 1 (got 4)

In [59]:
ids1= torch.tensor([[2026,2171,2003,10975,8988,3286]])

In [60]:
ids2 =torch.tensor([[1045,2444,1999,2634]])

In [73]:
all_ids = torch.tensor([[2026,2171,2003,10975,8988,3286],[1045,2444,1999,2634,0,0]])

In [74]:
from transformers import AutoModelForSequenceClassification

In [75]:

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
print(model(ids1).logits)
print(model(ids2).logits)
print(model(all_ids).logits)

tensor([[ 2.0751, -1.7523]], grad_fn=<AddmmBackward0>)
tensor([[-3.1949,  3.4589]], grad_fn=<AddmmBackward0>)
tensor([[ 2.0751, -1.7523],
        [-0.6553,  0.8964]], grad_fn=<AddmmBackward0>)


In [76]:
attention_mask = torch.tensor([[1,1,1,1,1,1],[1,1,1,1,0,0]])

In [77]:
output = model(all_ids,attention_mask=attention_mask)
print(output.logits)
#now we will get the same tensor as we would get if we pass the tokens indivudually

tensor([[ 2.0751, -1.7523],
        [-3.1949,  3.4589]], grad_fn=<AddmmBackward0>)
